In [1]:
import pyaudio
import struct
import numpy as np
from matplotlib import pyplot as plt

%matplotlib tk

In [2]:
import pprint

In [16]:
FORMAT = pyaudio.paInt16 # data type for our chunk size
CHANNELS = 1 # mono

CHUNK = 1024 * 4 # 4096 samples per chunk
RATE = 44100 # samples per second

In [4]:
p = pyaudio.PyAudio()

In [5]:
fig, ax = plt.subplots()

In [6]:
def unpack_data(data, size=CHUNK):
    return struct.unpack(str(size) + 'h', data)

In [7]:
default_device_index = 0
desired_dev_name = 'Steinberg UR22mkII'

chosen_device_index = default_device_index
chosen_device = p.get_device_info_by_index(default_device_index)

for dev_idx in range(0, p.get_device_count()):
    dev_info = p.get_device_info_by_index(dev_idx)
    
    if desired_dev_name in dev_info['name']:
        print('Found desired device!\n')        
        chosen_device_index = dev_idx
        chosen_device = dev_info
        break
        
pprint.pprint(chosen_device)

Found desired device!

{'defaultHighInputLatency': 0.1,
 'defaultHighOutputLatency': 0.014535147392290249,
 'defaultLowInputLatency': 0.01,
 'defaultLowOutputLatency': 0.004376417233560091,
 'defaultSampleRate': 44100.0,
 'hostApi': 0,
 'index': 3,
 'maxInputChannels': 2,
 'maxOutputChannels': 2,
 'name': 'Steinberg UR22mkII ',
 'structVersion': 2}


In [17]:
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input_device_index=chosen_device_index,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

#data_raw_bytes = stream.read(CHUNK)
#data_int = unpack_data(data_raw_bytes)

x = np.arange(0, CHUNK)
data = stream.read(CHUNK)
data_int16 = struct.unpack(str(CHUNK) + 'h', data)
line, = ax.plot(x, data_int16)
ax.set_ylim([-2**15, 2**15 - 1])

while True:
    try:
        raw_data = stream.read(CHUNK)
    except OSError as e:
        
        if pyaudio.paInputOverflowed != -9981:
            raise
        raw_data = b'\x00' * CHUNK
        
    data_int16 = unpack_data(raw_data)
    line.set_ydata(data_int16)
    fig.canvas.draw()

    fig.canvas.flush_events()
    fig.show()

error: unpack requires a buffer of 8192 bytes

In [ ]:
pyaudio.paInputOverflowed